# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, afternoon  
Session on working with `Activity` and ` Exchange` proxies 

### Learning objectives  
  - Work with `Activity` and ` Exchange` proxies (copy, add, modify, delete)  

**Exercise**: Create a new activity for blow moulding in the USA that is based on a copy of the activity `'stretch blow moulding`. It is to have the following modifications:  
  - Its location is to be changed to "US" ==> changing an activity
  - Its source of electricity is to be changed to medium voltage electricity from the US grid ==> changing an exchange  
  - The waste heat elementary flow is to be deleted ==> Deleting an exchange  
  - The cooling water is to come from the "river" subcompartment ==> Deleting and adding an exchange

CHRIS: this may be fun, but probably would take too long.
  - The Pedigree matrix scores for the exchanges are to be modified  
  - The uncertainty is to be modified based on these modified Pedigree scores

**Step 1**: Import Brightway as bw, open the seminar project and make a copy of the ecoinvent v2.2 database.

In [1]:
import brightway2 as bw

In [ ]:
bw.projects.set_current('bw2_seminar_2017')

In [ ]:
if 'ecoinvent 2.2 copy' not in bw.databases:
    bw.Database('ecoinvent 2.2').copy('ecoinvent 2.2 copy')
else:
    print('A copy already exists')
eidb = bw.Database('ecoinvent 2.2 copy')

**Step 2**: Create a copy of the activity `'offset printing, per kg printed paper' (kilogram, CH, None)` and assign it to a variable.  

In [ ]:
bm_RER = [act for act in eidb if 'stretch blow moulding' in act['name']][0]
bm_RER

In [ ]:
bm_US = bm_RER.copy()

Before we go on, let's get a sense of what the activity looks like and what exchanges are output into the activity.

In [ ]:
bm_US.as_dict()

In [ ]:
for exc in bm_US.exchanges():
    print(exc.input['name'], exc['amount'], exc['unit'])

**Step 3**: Just a check: make sure that the key of the activities are not the same

In [ ]:
bm_RER.key == bm_US.key

**Step 4**: Change the US activity's location to "US"

In [ ]:
bm_US['location']="US"
bm_US.save()

**Step 5**: Find the electricity exchange to be replaced in the US printing activity.

In [ ]:
electricity_input_exchange = [exc for exc in bm_US.technosphere() if "electricity" in exc['name']][0]
electricity_input_exchange

**Step 6**: Find the activity that this exchange needs to be relinked to.

In [ ]:
US_elect=[act for act in eidb if 'electricity, medium voltage' in act['name']
          and act['location']=='US'
         ][0]
US_elect

**Step 7**: Relink the electricity input exchange to the US electricity:

In [ ]:
electricity_input_exchange.input = US_elect

In [ ]:
for exc in bm_US.technosphere():
    if "electricity" in exc.input['name']:
        print(exc.input['name'], exc.input['location']) 

**Step 8**: Delete the waste heat elementary flow:

In [ ]:
waste_heat = [exc for exc in bm_US.biosphere() if "Heat, waste" in exc['name']][0]
waste_heat

In [ ]:
waste_heat.delete()

**Step 9**: Create a new exchange for the water input from river and delete the old cooling water, unspecified

In [ ]:
[ef for ef in bw.Database('biosphere3') if "river" in ef['name']]  # To determine how best to return my activity
river_water = [ef for ef in bw.Database('biosphere3') if "Water, river" in ef['name']][0]
river_water

In [ ]:
# Also return the existing cooling water flow to "steal" some information from it
old_cooling_water = [ef for ef in bm_US.biosphere() if "cooling" in ef['name']][0]
old_cooling_water

In [ ]:
old_cooling_water.as_dict()

In [ ]:
new_water = bm_US.new_exchange(input=river_water.key,
                   amount=old_cooling_water['amount'],
                   unit=old_cooling_water['unit'],
                   categories=old_cooling_water['categories'],
                   type='biosphere')

In [ ]:
new_water.save()

Then delete the old water input:

In [ ]:
old_cooling_water.delete()

In [ ]:
bm_US.save()

**Final result:** 

In [ ]:
for exc in bm_US.exchanges():
    try:
        print(exc.input['name'], exc['amount'], exc['unit'], exc.input['location'])
    except:
        print(exc.input['name'], exc['amount'], exc['unit'])